In [ ]:
!git clone --depth 1 https://github.com/Tianxiaomo/pytorch-YOLOv4

In [ ]:
# !pip install -U setuptools
# !apt install enchant
# !pip install pyenchant

In [ ]:
%cd /content/pytorch-YOLOv4/
!pip install -r requirements.txt

In [ ]:
# !pip3 install skimage
# !pip install scikit-image
!pip install tensorboardX

In [ ]:
!mkdir dataset;
# !unzip /content/task_garbage_detection_2-2020_07_22_12_17_37-yolo\ 1.1.zip -d ./dataset

# !unzip /content/drive/My\ Drive/20200722/task_garbage_detection_2-2020_07_22_12_17_37-yolo\ 1.1.zip -d ./dataset
# !unzip /content/drive/My\ Drive/20200722/fps_5_frames.zip -d  /content/pytorch-YOLOv4/dataset/obj_train_data/

!unzip /content/drive/My\ Drive/20200722/task_garbage_detection_2-2020_07_22_12_17_37-yolo\ 1.1.zip -d ./data
!unzip /content/drive/My\ Drive/20200722/fps_5_frames.zip -d  /content/pytorch-YOLOv4/data/obj_train_data/

In [ ]:
!python train.py -g 0 -dir /content/pytorch-YOLOv4/data

In [ ]:
# !cat /content/pytorch-YOLOv4/data/train.txt

# Darknet

In [ ]:
# based on https://colab.research.google.com/drive/1_GdoqCJWXsChrOiY8sZMr_zbr_fH-0Fg?usp=sharing#scrollTo=GQQrAMdXN22a
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [6]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
!unzip /content/drive/My\ Drive/20200722/task_garbage_detection_2-2020_07_23_13_39_54-yolo-1.1.zip -d /content/darknet/data/

In [9]:
!cp /content/drive/My\ Drive/20200722/yolov4-custom.cfg /content/darknet/cfg/

In [ ]:
!rm /content/backup/*

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

 try to allocate additional workspace_size = 52.43 MB 
 CUDA allocate done! 
Loaded: 0.000051 seconds
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.852891, GIOU: 0.849890), Class: 0.995506, Obj: 0.388808, No Obj: 0.003254, .5R: 0.986842, .75R: 0.855263, count: 76, class_loss = 10.775946, iou_loss = 131.666367, total_loss = 142.442307 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.897685, GIOU: 0.895843), Class: 0.998361, Obj: 0.480648, No Obj: 0.012198, .5R: 1.000000, .75R: 0.956522, count: 69, class_loss = 8.478806, iou_loss = 29.340504, total_loss = 37.819309 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.901919, GIOU: 0.901172), Class: 0.999343, Obj: 0.492098, No Obj: 0.004875, .5R: 1.000000, .75R: 1.000000, count: 6, class_loss = 0.819634, iou_loss = 0.574590, total_loss = 1.394224 
 total_bbox = 5023954, rewritten_bbox = 0.004857 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.

In [ ]:
imShow('chart.png')

In [ ]:
!cp cfg/yolov4-custom.cfg /content/drive/My\ Drive/20200722/yolov4-custom.cfg

In [ ]:
!mkdir /content/drive/My\ Drive/20200722/yolow_backup

In [ ]:
!cp /content/darknet/backup/* /content/drive/My\ Drive/20200722/yolow_backup

# Detections

### Import existing model

In [30]:
_data_path = 'data/obj.data'
_cfg_path = 'cfg/yolov4-custom.cfg'
_weight_path = '/content/drive/My\ Drive/20200722/yolow_backup/yolov4-custom_best.weights'

In [ ]:
img_path = '/content/fps_5_frame_0014.jpg'
# !./darknet detector map data/obj.data cfg/yolov4-custom.cfg /content/darknet/backup/yolov4-custom_last.weights {img_path} -thresh 0.3 -dont_show #-out /content/my_img.jpg
# !./darknet detector test data/obj.data cfg/yolov4-custom.cfg \
#         /content/darknet/backup/yolov4-custom_last.weights {img_path} \
#         -thresh 0.1 -dont_show #-out /content/my_img.jpg
!./darknet detector test data/obj.data cfg/yolov4-custom.cfg \
        {weight_path} {img_path} \
        -thresh 0.1 -dont_show #-out /content/my_img.jpg

imShow('predictions.jpg')

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-custom.cfg \
    /content/darknet/backup/yolov4-custom_last.weights -dont_show /content/VID_20200722_115436_stabiilizo_annot.mp4 -i 0 -out_filename results.avi -thresh 0.1

In [ ]:
!cp ./results.avi /content/drive/My\ Drive/20200722/

In [ ]:
!cp data/obj.data /content/drive/My\ Drive/20200722/yolow_backup
!cp cfg/yolov4-custom.cfg /content/drive/My\ Drive/20200722/yolow_backup

# Generate annotations

In [16]:
import os
import shutil

## Convert Video to frames and create frame_list.txt 

In [22]:
%%capture
# Video to frames
frame_dir = '/content/frames'
input_video = '/content/VID_20200722_115436_stabiilizo_annot.mp4'
# clear folder if exsists
if os.path.isdir(frame_dir):
    shutil.rmtree(frame_dir)    
os.mkdir(frame_dir)

fps = 15
frames_name = os.path.join(frame_dir, "fps_{}_frame_%04d.jpg".format(fps))
!ffmpeg -i {input_video} -vf fps={fps} -qscale:v 2 {frames_name}

In [ ]:
from os import listdir
from os.path import isfile, join

frame_list_file = '/content/frames_list.txt'

frame_list = [join(frame_dir, f) for f in listdir(frame_dir) if isfile(join(frame_dir, f))]
print(len(frame_list))
# print(frame_list)

with open(frame_list_file, 'w') as f:
    for frame_path in frame_list:
        f.write(frame_path + "\n")

In [ ]:
!./darknet detector test {_data_path} {_cfg_path} {_weight_path} -dont_show -out /content/result.json -ext_output <{frame_list_file}> /content/result.txt

## Convert JSON to Pascal VOC

In [58]:
!pip install pascal-voc-writer

In [62]:
import json
from pprint import pprint
from PIL import Image
from pascal_voc_writer import Writer

In [63]:
with open('/content/result.json') as f:
    data = json.load(f)
# pprint(data)

In [ ]:
# # Writer(path, width, height)
# writer = Writer('path/to/img.jpg', 800, 400)

# # ::addObject(name, xmin, ymin, xmax, ymax)
# writer.addObject('cat', 100, 100, 200, 200)

# # ::save(path)
# writer.save('path/to/img.xml')

In [81]:

dir = '/content/pascal_voc'
for det_result in data:
    # pprint(image_res)
    # writer = Writer('path/to/img.jpg', 800, 400)

    img_path = det_result['filename']
    img = Image.open(img_path)
    width, height = img.size

    # Writer(path, width, height)
    writer = Writer(img_path, width, height)

    for obj in det_result['objects']:
        bb_x_center = obj['relative_coordinates']['center_x']
        bb_y_center = obj['relative_coordinates']['center_y']
        
        bb_width = obj['relative_coordinates']['width']
        bb_height = obj['relative_coordinates']['height']

        xmin = int((bb_x_center - bb_width/2) * width)
        xmax = int((bb_x_center + bb_width/2) * width)
        
        ymin = int((bb_y_center - bb_height/2) * height)
        ymax = int((bb_y_center + bb_height/2) * height)

        # ::addObject(name, xmin, ymin, xmax, ymax)
        writer.addObject(obj['name'], xmin, ymin, xmax, ymax)

    #image name without extension
    xml_name = img_path.split("/")[-1].split('.')[0] + ".xml"
    xml_path = os.path.join(dir, xml_name)
    writer.save(xml_path)
    # print(xml_name)

In [ ]:
!cd /content/; zip frames/pascal_voc.zip pascal_voc/*
!cd /content/; zip garbage_det_fps_15_img_annot_322.zip frames/*

In [86]:
!cp /content/garbage_det_fps_15_img_annot_322.zip /content/drive/My\ Drive/20200722